# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686579


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:02<00:29,  1.00s/it]

Rendering models:  10%|▉         | 3/31 [00:04<00:44,  1.59s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:53,  1.97s/it]

Rendering models:  16%|█▌        | 5/31 [00:08<00:39,  1.52s/it]

...

Rendering models:  87%|████████▋ | 27/31 [00:13<00:00,  4.59it/s]

Rendering models:  90%|█████████ | 28/31 [00:13<00:00,  4.50it/s]

Rendering models:  97%|█████████▋| 30/31 [00:13<00:00,  5.83it/s]

Rendering models: 100%|██████████| 31/31 [00:13<00:00,  5.62it/s]

not-logged-in-f6949d2cd1af58cfa627    0.000255
not-logged-in-20a275b53abec8db9f41    0.012930
RockyGeo                              0.000108
CThomas                               0.000253
not-logged-in-db23dc032fc427a4e4d9    0.000122
not-logged-in-f5ac928c4dcd4e9cb619    0.322801
not-logged-in-13bceff57f1f879f68d7    0.000129
not-logged-in-c3e2bb63f9ec3bdeea91    0.000465
mmcdonough4                           0.000289
not-logged-in-a5721f5f24fded19f7ed    0.000522
elsawinslow                           0.000285
not-logged-in-41e9fe197dc7a2be8424    0.000372
Jessica_Korkmaz                       0.009438
Dylano                                0.000397
Lavadude                              0.062904
ShirleyL                              0.000152
astronautlady                         0.000303
Theshoestargazers                     0.009112
IdeasInSpace                          0.000140
jthrasher                             0.000298
not-logged-in-3a93234a2311f6df1afd    0.000178
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())